In [1]:
import os
from ultralytics import NAS
os.chdir("..")
ROOT = os.getcwd()
print(ROOT)


/Users/mautushid/github/Cowsformer


In [2]:
from models.nas import *

[2023-10-19 09:16:00] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /Users/mautushid/sg_logs/console.log


[2023-10-19 09:16:00] WARNING - redirects.py - NOTE: Redirects are currently not supported in Windows or MacOs.
[2023-10-19 09:16:02] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-10-19 09:16:02] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-10-19 09:16:02] WARNING - export.py - Failed to import pytorch_quantization
[2023-10-19 09:16:02] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2023-10-19 09:16:02] WARNING - env_sanity_check.py - Failed to verify operating system: Deci officially supports only Linux kernels. Some features may not work as expected.


In [4]:
### Creating instance of Niche_YOLO_NAS class
path_model = 'yolo_nas_l' #"/Users/mautushid/github/Cowsformer/n200_yolo_na_i1_trial100/ckpt_best.pth"
dir_train = "/Users/mautushid/github/Cowsformer/data/cow200/yolov5/train"
dir_val = "/Users/mautushid/github/Cowsformer/data/cow200/yolov5/val"
dir_test = "/Users/mautushid/github/Cowsformer/data/cow200/yolov5/test"
name_task = "cow200"
my_nas = Niche_YOLO_NAS(path_model, dir_train, dir_val, dir_test, name_task)


UnknownTypeException: The required model, "/Users/mautushid/github/Cowsformer/n200_yolo_na_i1_trial100/ckpt_best.pth", was not found in SuperGradients. See docs or all_architectures.py for supported model names.
 Did you mean: yolo_nas_s?

In [11]:
yaml_path = "/Users/mautushid/github/Cowsformer/data/cow200/yolov5/data.yaml"
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

CLASSES = data.get('names', [])

CHECKPOINT_DIR = '/Users/mautushid/github/Cowsformer/n200_yolo_na_i1_trial100'
project = os.path.basename(os.path.dirname(os.path.dirname(yaml_path)))
# Format the project name
EXPERIMENT_NAME = project.lower().replace(" ", "_")


In [9]:
trainer = Trainer(experiment_name=EXPERIMENT_NAME,
                  ckpt_root_dir=CHECKPOINT_DIR)
MODEL_ARCH = 'yolo_nas_l'
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
best_model = models.get(
    MODEL_ARCH,
    num_classes=80,  # len(dataset_params['classes']),
    checkpoint_path="/Users/mautushid/github/Cowsformer/n200_yolo_na_i1_trial100/ckpt_best.pth"
).to(DEVICE)


[2023-10-19 08:16:27] INFO - checkpoint_utils.py - Successfully loaded model weights from /Users/mautushid/github/Cowsformer/n200_yolo_na_i1_trial100/ckpt_best.pth EMA checkpoint.


TypeError: __init__() missing 5 required positional arguments: 'path_model', 'dir_train', 'dir_val', 'dir_test', and 'name_task'

In [12]:
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train, coco_detection_yolo_format_val)
dataset_params = {
    'data_dir': '/Users/mautushid/github/Cowsformer/data/cow200/yolov5',
    'train_images_dir': 'train/images',
    'train_labels_dir': 'train/labels',
    'val_images_dir': 'val/images',
    'val_labels_dir': 'val/labels',
    'test_images_dir': 'test/images',
    'test_labels_dir': 'test/labels',
    'classes': CLASSES
}
########################################

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': 16,
        'num_workers': 2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': 16,
        'num_workers': 2
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': 16,
        'num_workers': 2
    }
)


Caching annotations:   0%|          | 0/45 [00:00<?, ?it/s]

Caching annotations: 100%|██████████| 45/45 [00:00<00:00, 260.12it/s]


In [19]:
#best_model
trainer.test(
    model=best_model,
    test_loader=test_data,
    test_metrics_list=DetectionMetrics_050(
        score_thres=0.5,
        top_k_predictions=300,
        num_cls=len(dataset_params['classes']),
        normalize_targets=True,
        post_prediction_callback=PPYoloEPostPredictionCallback(
            score_threshold=0.5,
            nms_top_k=1000,
            max_predictions=300,
            nms_threshold=0.8
        )
    )
)


Test:   0%|          | 0/3 [00:00<?, ?it/s][W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
Test: 100%|██████████| 3/3 [01:19<00:00, 22.96s/it]

{'Precision@0.50': tensor(0.8243),
 'Recall@0.50': tensor(0.9683),
 'mAP@0.50': tensor(0.9506),
 'F1@0.50': tensor(0.8905)}

Test: 100%|██████████| 3/3 [01:29<00:00, 29.79s/it]
